In [1]:
from typing import Final

import numpy as np

In [2]:
RANDOM_SEED:  Final[int]    = 42
TEST_SIZE:    Final[float]  = 0.2

np.random.seed(RANDOM_SEED)

In [3]:
def entropy(y):
    """
    Berechnet die Entropie eines gegebenen Datensatzes y.

    Args:
        y (array-like): Ein Array oder eine Liste von Zielwerten (Labels)

    Returns:
        float: Die Entropie des Datensatzes y.

    Beispiel:
        entropy([1, 1, 0, 0]) → 1.0
    """
    _, counts = np.unique(y, return_counts=True)
    prob = counts / len(y)

    # y = H(V) = -Σ P(Bj) * log2(P(Bj))
    return -np.sum(prob * np.log2(prob))


def information_gain(X, y, feature_index):
    """
    Berechnet den Informationsgewinn für ein bestimmtes Feature im Datensatz.

    Args:
        X (array-like): Das Array der Eingabefeatures
        y (array-like): Die Labels, die mit X verknüpft sind.
        feature_index (int): Der Index des Features, für das der Informationsgewinn berechnet wird.

    Returns:
        float: Der Informationsgewinn für das angegebene Feature.

    Beispiel:
        information_gain(X, y, 0) → 0.5
    """
    parent_entropy = entropy(y)
    unique_values = np.unique(X[:, feature_index])

    weighted_entropy = 0
    for value in unique_values:
        subset_y = y[X[:, feature_index] == value]

        # P(Ai | Bj) * H(V | Bj)
        weighted_entropy += (len(subset_y) / len(y)) * entropy(subset_y)

    # V = Zielvariable, W = Feature
    # IG(V, W) = H(V) - H(V|W)
    return parent_entropy - weighted_entropy


def best_split(X, y):
    """
    Bestimmt das beste Feature für den Split, basierend auf dem maximalen Informationsgewinn.

    Args:
        X (array-like): Das Array der Eingabefeatures.
        y (array-like): Das Ziel-Array (Labels), das mit X verknüpft ist.

    Returns:
        tuple: Ein Tupel, das den Index des besten Features und den maximalen Informationsgewinn enthält.

    Beispiel:
        best_split(X, y) → (0, 0.25)
    """
    best_feature_index = None
    best_info_gain     = -1

    for feature_index in range(X.shape[1]):
        info_gain = information_gain(X, y, feature_index)
        if info_gain > best_info_gain:
            best_info_gain     = info_gain
            best_feature_index = feature_index

    return best_feature_index, best_info_gain

In [4]:
class MyDecisionTree:
    """
    Ein einfacher Entscheidungsbaum-Algorithmus für binäre Klassifikation.

    Args:
        max_depth (int): Die maximale Tiefe des Entscheidungsbaums. Wenn nicht angegeben, wird der Baum so tief wie nötig gebaut.

    Methoden:
        - fit(X, y): Trainiert den Entscheidungsbaum mit den Eingabedaten X und den Zielwerten y.
        - predict(X): Gibt die vorhergesagten Labels für die Eingabedaten X zurück.

    Beispiel:
        tree = MyDecisionTree(max_depth=3)
        tree.fit(X_train, y_train)
        y_pred = tree.predict(X_test)
    
    Referenzen:
        - Vorlesungsfolien: ML_VL_9_Entscheidungsbaeume.pdf 
        - https://www.analyticsvidhya.com/blog/2020/10/all-about-decision-tree-from-scratch-with-python-implementation/
        - https://medium.com/@omidsaghatchian/decision-tree-implementation-from-scratch-visualization-5eb0bbf427c2
        - https://www.kaggle.com/code/fareselmenshawii/decision-tree-from-scratch
    """
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        # Wenn alle Beispiele in der gleichen Klasse sind, ist dies ein Blatt
        if len(np.unique(y)) == 1:
            return {"label": np.unique(y)[0]}

        # Wenn keine weiteren Splits möglich sind oder die max. Tiefe erreicht ist, ist dies ein Blatt
        if len(X) == 0 or (self.max_depth and depth >= self.max_depth):
            return {"label": np.bincount(y).argmax()}

        feature_index, info_gain = best_split(X, y)

        # Wenn der Information Gain 0 ist, ist dies auch ein Blatt (Nicht mehr sinnvoll zu splitten)
        if info_gain == 0:
            return {"label": np.bincount(y).argmax()}


        unique_values = np.unique(X[:, feature_index])
        tree = {"feature_index": feature_index, "branches": {}}

        for value in unique_values:
            subset_X = X[X[:, feature_index] == value]
            subset_y = y[X[:, feature_index] == value]
            tree["branches"][value] = self._build_tree(subset_X, subset_y, depth + 1)

        return tree

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])

    def _predict_one(self, x, tree):
        if "label" in tree:
            return tree["label"]

        feature_value = x[tree["feature_index"]]
        branch = tree["branches"].get(feature_value)

        if branch is None:
            global y_train
            return np.bincount(y_train).argmax()

        return self._predict_one(x, branch)

In [5]:
class MyRandomForest:
    """
    Ein einfacher Random Forest-Algorithmus für binäre Klassifikation.

    Args:
        n_estimators (int): Die Anzahl der Bäume.
        max_depth (int): Die maximale Tiefe eines jeden Entscheidungsbaums.
        sample_size (float): Der Prozentsatz der Trainingsdaten, die für jeden Baum verwendet werden.

    Methoden:
        - fit(X, y): Trainiert den RandomForest mit den Eingabedaten X und den Labels y.
        - predict(X): Gibt die vorhergesagten Labels für die Eingabedaten X zurück.

    Beispiel:
        forest = MyRandomForest(n_estimators=100, max_depth=5, sample_size=0.5)
        forest.fit(X_train, y_train)
        y_pred = forest.predict(X_test)

    Referenzen:
        - Vorlesungsfolien: ML_VL_9_Entscheidungsbaeume.pdf & ML_VL_10_EnsembleLearning.pdf
        - https://towardsdatascience.com/random-forests-and-decision-trees-from-scratch-in-python-3e4fa5ae4249
        - https://www.kaggle.com/code/fareselmenshawii/random-forest-from-scratch
    """
    def __init__(self, n_estimators=10, max_depth=None, sample_size=1.0):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.sample_size = sample_size
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = int(len(X) * self.sample_size)
        indices = np.random.choice(len(X), size=n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            tree = MyDecisionTree(max_depth=self.max_depth)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        majority_votes = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=tree_predictions)
        return majority_votes


In [6]:
def my_train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)

    n_samples = len(X)
    test_size = int(n_samples * test_size)
    indices   = np.random.permutation(n_samples)

    test_indices  = indices[:test_size]
    train_indices = indices[test_size:]

    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    return X_train, X_test, y_train, y_test


def my_accuracy_score(y_true, y_pred):
    correct_predictions = np.sum(np.equal(y_true, y_pred))
    total_predictions   = len(y_true)

    return correct_predictions / total_predictions

In [7]:
def my_classification_report(y_true, y_pred):
    unique_labels = np.unique(y_true)
    report = {}

    for label in unique_labels:
        true_positive = np.sum(np.logical_and(y_true == label, y_pred == label))
        false_positive = np.sum(np.logical_and(y_true != label, y_pred == label))
        false_negative = np.sum(np.logical_and(y_true == label, y_pred != label))

        precision = true_positive / (true_positive + false_positive) if true_positive + false_positive > 0 else 0
        recall = true_positive / (true_positive + false_negative) if true_positive + false_negative > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        report[label] = {"precision": precision, "recall": recall, "f1-score": f1_score}

    return report

def report_to_string(report: dict):
    if 'precision' not in report[next(iter(report))] or 'recall' not in report[next(iter(report))] or 'f1-score' not in report[next(iter(report))]:
        return "Precision, Recall and F1-Score are required in the report."

    report_str = ""
    for label, metrics in report.items():
        report_str += f"Label {label}:\n"
        report_str += f"  Precision: {metrics['precision']:.2f}\n"
        report_str += f"  Recall:    {metrics['recall']:.2f}\n"
        report_str += f"  F1-Score:  {metrics['f1-score']:.2f}\n"
    return report_str

In [8]:
from sklearn import datasets

X, y =  datasets.load_iris(return_X_y=True, as_frame=True)

X_numpy = X.to_numpy()
y_numpy = y.to_numpy()

X_train, X_test, y_train, y_test = my_train_test_split(X_numpy, y_numpy, test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [9]:
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [10]:
forest = MyRandomForest(n_estimators=5, max_depth=3, sample_size=0.8)
forest.fit(X_train, y_train)
forest_predictions = forest.predict(X_test)

In [11]:
report = my_classification_report(y_test, forest_predictions)
print(report_to_string(report))

Label 0:
  Precision: 1.00
  Recall:    1.00
  F1-Score:  1.00
Label 1:
  Precision: 0.56
  Recall:    1.00
  F1-Score:  0.72
Label 2:
  Precision: 1.00
  Recall:    0.36
  F1-Score:  0.53



In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

sklearn_forest = RandomForestClassifier(n_estimators=5, max_depth=3, random_state=RANDOM_SEED)
sklearn_forest.fit(X_train, y_train)
sklearn_predictions = sklearn_forest.predict(X_test)

print(classification_report(y_test, sklearn_predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      1.00      0.95         9
           2       1.00      0.91      0.95        11

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

